# Homework for Module 3

Date: 2025. 10. 19


By Jayon Lihm


## Data prep

In [1]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## Data preparation
Check if the missing values are presented in the features.

If there are missing values:

For categorical features, replace them with 'NA'

For numerical features, replace with with 0.0

In [3]:
df = pd.read_csv("./course_lead_scoring.csv")

In [4]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [5]:
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [6]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [7]:
categorical = list(df.dtypes[df.dtypes=="object"].index)
numerical = list(df.dtypes[df.dtypes!="object"].index)

In [8]:
df[categorical].head()

,lead_source,industry,employment_status,location
0,paid_ads,NaN,unemployed,south_america
1,social_media,retail,employed,south_america
2,events,healthcare,unemployed,australia
3,paid_ads,retail,NaN,australia
4,referral,education,self_employed,europe


In [9]:
df[numerical].head()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
0,1,79450.0,4,0.94,1
1,1,46992.0,1,0.80,0
2,5,78796.0,3,0.69,1
3,2,83843.0,1,0.87,0
4,3,85012.0,3,0.62,1


In [10]:
## check missing values
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [11]:
## categorical variable processing
for c in categorical:
    df[c] = df[c].str.lower().str.replace(' ', '_').fillna('NA')

In [12]:
df[categorical].head()

,lead_source,industry,employment_status,location
0,paid_ads,NA,unemployed,south_america
1,social_media,retail,employed,south_america
2,events,healthcare,unemployed,australia
3,paid_ads,retail,NA,australia
4,referral,education,self_employed,europe


In [13]:
## numerical variable processing
for n in numerical:
    df[n] = df[n].fillna(0)

In [14]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

## Question 1 - retail

What is the most frequent observation (mode) for the column `industry`?

* NA

* technology

* healthcare

* retail


In [15]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

## Question 2 - annual_income and interaction_count

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

* interaction_count and lead_score

* number_of_courses_viewed and lead_score

* number_of_courses_viewed and interaction_count

* annual_income and interaction_count


Only consider the pairs above when answering this question.

In [16]:
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [17]:
df[numerical].corr().loc[ "interaction_count", "lead_score"]

0.009888182496913105

In [18]:
df[numerical].corr().loc[ "number_of_courses_viewed", "lead_score"]

-0.004878998354681265

In [19]:
df[numerical].corr().loc[ "number_of_courses_viewed", "interaction_count"]

-0.023565222882888055

In [20]:
df[numerical].corr().loc[ "annual_income", "interaction_count"]

0.027036472404814396

## Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.

* Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.

* Make sure that the target value converted is not in your dataframe.

In [71]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42) 
df_full_train.shape, df_test.shape

((1169, 9), (293, 9))

In [72]:
df_train, df_val = train_test_split(df_full_train, test_size=0.2/0.8, random_state=42)
df_train.shape, df_val.shape, df_test.shape

((876, 9), (293, 9), (293, 9))

In [73]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

## Question 3 - lead_source

Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.

Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

* industry
* location
* lead_source
* employment_status

In [23]:
from sklearn.metrics import mutual_info_score

In [74]:
mi_score={}
for c in categorical:
    mi_score[c]= round(mutual_info_score(df_train.converted, df_train[c]), 2)

In [75]:
mi_score

{'lead_source': 0.04,
 'industry': 0.01,
 'employment_status': 0.01,
 'location': 0.0}

## Question 4 - 0.74

Now let's train a logistic regression. 

Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.

Fit the model on the training dataset.

To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:

`model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`

Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

* 0.64
* 0.74
* 0.84
* 0.94

### One hot coding

In [76]:
del df_train['converted']
del df_val['converted']
del df_test['converted']
del df_full_train['converted']

In [27]:
from sklearn.feature_extraction import DictVectorizer

In [77]:
train_dicts = df_train.to_dict(orient='records')
val_dicts = df_val.to_dict(orient='records')
test_dicts = df_test.to_dict(orient='records')

In [84]:
dv = DictVectorizer(sparse = False)
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)
X_test = dv.transform(test_dicts)

### Logistic Regression modeling

In [85]:
from sklearn.linear_model import LogisticRegression

In [86]:
#model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model = LogisticRegression(C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [87]:
y_val_prob = model.predict_proba(X_val)[:, 1]
y_val_pred = model.predict(X_val)

In [88]:
#conv_decision = (y_val_prob >= 0.5)
conv_decision = y_val_pred

In [89]:
round((y_val == conv_decision.astype(int)).mean(), 2)

0.7

In [60]:
(y_val == conv_decision.astype(int)).mean()

0.6996587030716723

## Question 5 - industry

Let's find the least useful feature using the feature elimination technique.

Train a model using the same features and parameters as in Q4 (without rounding).

Now exclude each feature from this set and train a model without it. Record the accuracy for each model.

For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

* 'industry'
* 'employment_status'
* 'lead_score'

Note: The difference doesn't have to be positive.

In [52]:
## remove one column and train model again
accuracy_scores = {}
for remove in ['industry', 'employment_status', 'lead_score']:
    train_dicts_r = df_train.drop([remove], axis=1).to_dict(orient='records')
    val_dicts_r = df_val.drop([remove], axis=1).to_dict(orient='records')
    test_dicts_r = df_test.drop([remove], axis=1).to_dict(orient='records')
    dv = DictVectorizer(sparse = False)
    X_train_r = dv.fit_transform(train_dicts_r)
    X_val_r = dv.fit_transform(val_dicts_r)
    X_test_r = dv.fit_transform(test_dicts_r)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_r, y_train)
    y_val_prob_r = model.predict_proba(X_val_r)[:, 1]
    conv_decision = ( y_val_prob_r >= 0.5 )
    accuracy_scores[remove] = (y_val == conv_decision.astype(int)).mean()



In [53]:
accuracy_scores

{'industry': 0.6996587030716723,
 'employment_status': 0.6962457337883959,
 'lead_score': 0.7064846416382252}

In [61]:
original_accuracy = (y_val == conv_decision.astype(int)).mean()
original_accuracy

0.6996587030716723

In [62]:
for remove in ['industry', 'employment_status', 'lead_score']:
    diff = original_accuracy - accuracy_scores[remove]
    print(remove, diff)

industry 0.0
employment_status 0.0034129692832763903
lead_score -0.0068259385665528916


## Question 6 - 0.01
Now let's train a regularized logistic regression.

Let's try the following values of the parameter C: `[0.01, 0.1, 1, 10, 100]`.

Train models using all the features as in Q4.

Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

* 0.01
* 0.1
* 1
* 10
* 100

Note: If there are multiple options, select the smallest C.




In [65]:
C_accuracy_scores = {}
for c_value in [0.01, 0.1, 1, 10, 100]:
    model = LogisticRegression(solver='liblinear', C=c_value, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_val_prob_c = model.predict_proba(X_val)[:, 1]
    conv_decision = ( y_val_prob_c >= 0.5 )
    C_accuracy_scores[str(c_value)] = (y_val == conv_decision.astype(int)).mean()

C_accuracy_scores

{'0.01': 0.6996587030716723,
 '0.1': 0.6996587030716723,
 '1': 0.6996587030716723,
 '10': 0.6996587030716723,
 '100': 0.6996587030716723}

In [90]:
########## END ################